In [1]:
import pandas as pd;
import numpy as np;
import glob;
import operator;
from Bio import SeqIO

Read in virus sequence

In [2]:
fn_virusgenome = "data/NC_045512.2.fasta.fa";
for record in SeqIO.parse (fn_virusgenome, "fasta"):
    viruslength = len (record.seq);
    print ("Read in virus %s with sequence length %d" % (record.id, viruslength));

Read in virus NC_045512.2_Severe_acute_respiratory_syndrome_coronavirus_2_isolate_Wuhan-Hu-1,_complete_genome with sequence length 29903


Read in binding-site predictions

In [3]:
fns_predictions_sense = glob.glob("data/rna22v2-predictions/sense/*.txt");
fns_predictions_revcomp = glob.glob("data/rna22v2-predictions/revcomp/*.txt");
data_predictions_sense = pd.concat(map(lambda file: pd.read_csv(file, sep="\t", header=None), fns_predictions_sense), ignore_index=True)
data_predictions_revcomp = pd.concat(map(lambda file: pd.read_csv(file, sep="\t", header=None), fns_predictions_revcomp), ignore_index=True)

Add 1-indexed start/end coordinates columns from predictions into the pandas data frame.  Importantly, the negative-sense prediction coordinates will be converted so that all coordinates are relative to the original positive-sense strand.

In [4]:
COLUMNS_SEQSPAN = 2;
COLUMNS_SEQSPAN_SPLIT_START = 2;
COLUMNS_SEQSPAN_SPLIT_END = 3;
COLUMNS_ADDSTART = 'StartCoord';
COLUMNS_ADDEND = 'EndCoord';

# add start/end for sense predictions
coords = data_predictions_sense[COLUMNS_SEQSPAN].str.split (pat='_');
data_predictions_sense[COLUMNS_ADDSTART] = [int (myindex[COLUMNS_SEQSPAN_SPLIT_START]) for myindex in coords]
data_predictions_sense[COLUMNS_ADDEND] = [int (myindex[COLUMNS_SEQSPAN_SPLIT_END]) for myindex in coords]

# add start/end for negative-sense predictions. Negative-sense prediction coordinates will be converted so that all coordinates are relative to the original positive-sense strand
coords = data_predictions_revcomp[COLUMNS_SEQSPAN].str.split (pat='_');
data_predictions_revcomp[COLUMNS_ADDEND] = [(viruslength - int (myindex[COLUMNS_SEQSPAN_SPLIT_START]) + 1) for myindex in coords]
data_predictions_revcomp[COLUMNS_ADDSTART] = [(viruslength - int (myindex[COLUMNS_SEQSPAN_SPLIT_END]) + 1) for myindex in coords]

Find intersections between insilico target predictions and annotated genome regions

In [5]:
# return True if there is at least 1nt between coordinate sets A and B
def check_any_overlap (a_start, a_end, b_start, b_end):
    if (b_start >= a_start and b_start <= a_end):
        return True;
    if (b_end >= a_start and b_end <= a_end):
        return True;
    return False;

In [6]:
# read in virus features file
fn_virusfeatures = "data/WHCV_features.txt";
data_virusfeatures = pd.read_csv(fn_virusfeatures, sep="\t", header=0)

In [7]:
data_predictions_sense["strand"] = 'positive-sense';
data_predictions_revcomp["strand"] = 'negative-sense';

for index, virusfeaturerow in data_virusfeatures.iterrows ():
        virusfeaturestart = virusfeaturerow['Start'];
        virusfeatureend = virusfeaturerow['End'];
                
        newLogitFeatureColumn = "%s|%s" % (virusfeaturerow['Category'], virusfeaturerow['Subcategory']);
        # positive-sense
        colresult = map(lambda predrow: check_any_overlap (virusfeaturestart, virusfeatureend, predrow[1]['StartCoord'], predrow[1]['EndCoord']), data_predictions_sense.iterrows ())
        data_predictions_sense[newLogitFeatureColumn] = list (colresult);
        
        # negative-sense
        colresult = map(lambda predrow: check_any_overlap (virusfeaturestart, virusfeatureend, predrow[1]['StartCoord'], predrow[1]['EndCoord']), data_predictions_revcomp.iterrows ())
        data_predictions_revcomp[newLogitFeatureColumn] = list (colresult);



Filter miR's based on desired attributes

In [8]:
data_bothstrands = pd.concat ([data_predictions_sense, data_predictions_revcomp], axis=0, ignore_index=True);

scoredcolumnnames = [];
for index, virusfeaturerow in data_virusfeatures.iterrows ():
    
    # these are the features we'll include scores for
    if (virusfeaturerow['Category'] == 'FULLGENOME' or
        virusfeaturerow['Category'] == 'GENE' or
        virusfeaturerow['Category'] == 'UTR'):
        
        newLogitFeatureColumn = "%s|%s" % (virusfeaturerow['Category'], virusfeaturerow['Subcategory']);
        scoredcolumnnames.append (newLogitFeatureColumn);
        
mirscores = {};
for _, datarow in data_bothstrands.iterrows ():
    if (datarow[scoredcolumnnames].sum() > 0):
        if mirscores.get (datarow[0]) == None:
            mirscores[datarow[0]] = 1;
        else:
            mirscores[datarow[0]] = mirscores[datarow[0]] + 1;
        

In [9]:
sorted_mirscores = sorted(mirscores.items(), key=operator.itemgetter(1), reverse=True)
scores = list (mirscores.values ());
minvalue = 1; # must have at least 1 intersection
survivedmirs = [k for k,v in mirscores.items() if int(v) >= minvalue]

In [10]:
# read in miR coordinates (which are already sorted)
fn_sortedmircoords = "data/mature.coords.sorted.dupsallowed.mirbase22.5cols.txt";
data_sortedmircoords = pd.read_csv(fn_sortedmircoords, sep="\t", header=None)

In [11]:
col_coordfile_mirname = 4; # 0-indexed column that includes the miR name inside fn_sortedmircoords 
col_coordfile_chrom = 0;
col_coordfile_start = 2;

clusterddistance = 10000; # tag miR's if they are within this many NT from the start site of the previous regardless of strand
prevmir_chrom = None;
premir_start = None;

clusternum = 0;
clusteroutput = {};
emptyclusterrecord = { "uniquelocations": 0,
                       "numincluster": 0,
                       "details": ""
                     };

for index, sortedmircoords in data_sortedmircoords.iterrows():
    for survivedmir in survivedmirs:
        if survivedmir == sortedmircoords[col_coordfile_mirname]:
            appendtext = "";
            
            if ((sortedmircoords[col_coordfile_chrom] == prevmir_chrom) and
               (sortedmircoords[col_coordfile_start] - premir_start <= clusterddistance)):
        
                appendtext = "POTENTIAL CLUSTER: this miR is %d distance from the previously listed miR\n" % (sortedmircoords[col_coordfile_start] - premir_start);
            else:
                clusternum += 1;
                clusteroutput[clusternum] = dict (emptyclusterrecord);

            clusteroutput[clusternum]["numincluster"] += 1;
            clusteroutput[clusternum]["uniquelocations"] += mirscores[survivedmir];
            appendtext += "miR: %s (total uniq predicted binding locations: %d)\n" % (survivedmir, mirscores[survivedmir]);
            
            matchmir = data_bothstrands.loc[(data_bothstrands[0] == survivedmir) & (data_bothstrands["strand"] == 'positive-sense'), scoredcolumnnames];
            nonzeros = matchmir.sum()>0;
            if (sum (nonzeros) > 0):
                appendtext += "Positive-Sense: \n";
                appendtext += matchmir.sum()[nonzeros].to_string ();
                appendtext += "\n\n";

            matchmir = data_bothstrands.loc[(data_bothstrands[0] == survivedmir) & (data_bothstrands["strand"] == 'negative-sense'), scoredcolumnnames];    
            nonzeros = matchmir.sum()>0;
            if (sum (nonzeros) > 0):
                appendtext += "Negative-Sense: \n";
                appendtext += matchmir.sum()[nonzeros].to_string ();
                appendtext += "\n\n";
                
            clusteroutput[clusternum]["details"] += appendtext;
            
            prevmir_chrom = sortedmircoords[col_coordfile_chrom];
            premir_start = sortedmircoords[col_coordfile_start];

In [15]:
# sort hash in desending order by uniquelocations & numincluster
sortedoutput = [v for k, v in sorted(clusteroutput.items(), key=lambda item: (-item[1]["uniquelocations"], -item[1]["numincluster"]))]

for sortoutput in sortedoutput:
    print ("---------------- Cluster Start -----------------------------------")
    print ("unique_predicted_target_locations: %d" % (sortoutput["uniquelocations"]))
    print ("number_of_miRNAs_in_cluster: %d" % (sortoutput["numincluster"]))
    print ("%s" % (sortoutput["details"]))
    print ();


---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 450
number_of_miRNAs_in_cluster: 67
miR: hsa_miR_379_5p (total uniq predicted binding locations: 5)
Positive-Sense: 
FULLGENOME|fullspan    1
GENE|1AB               1

Negative-Sense: 
FULLGENOME|fullspan    4
GENE|1AB               3
GENE|S                 1

POTENTIAL CLUSTER: this miR is 38 distance from the previously listed miR
miR: hsa_miR_379_3p (total uniq predicted binding locations: 1)
Positive-Sense: 
FULLGENOME|fullspan    1
GENE|1AB               1

POTENTIAL CLUSTER: this miR is 1231 distance from the previously listed miR
miR: hsa_miR_411_5p (total uniq predicted binding locations: 2)
Negative-Sense: 
FULLGENOME|fullspan    2
GENE|1AB               1
GENE|S                 1

POTENTIAL CLUSTER: this miR is 35 distance from the previously listed miR
miR: hsa_miR_411_3p (total uniq predicted binding locations: 2)
Positive-Sense: 
FULLGENOME|fullspan    2
GENE|1AB          

---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 16
number_of_miRNAs_in_cluster: 2
miR: hsa_miR_320a_3p (total uniq predicted binding locations: 10)
Positive-Sense: 
FULLGENOME|fullspan    5
GENE|1AB               4
GENE|S                 1

Negative-Sense: 
FULLGENOME|fullspan    5
GENE|1AB               5

POTENTIAL CLUSTER: this miR is 32 distance from the previously listed miR
miR: hsa_miR_320a_5p (total uniq predicted binding locations: 6)
Positive-Sense: 
FULLGENOME|fullspan    5
GENE|1AB               4
GENE|N                 1

Negative-Sense: 
FULLGENOME|fullspan    1
GENE|1AB               1



---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 16
number_of_miRNAs_in_cluster: 2
miR: hsa_miR_6876_5p (total uniq predicted binding locations: 6)
Positive-Sense: 
FULLGENOME|fullspan    4
GENE|1AB               1
GENE|S                 1
GENE|3a                1
GENE|N           

number_of_miRNAs_in_cluster: 1
miR: hsa_miR_7515 (total uniq predicted binding locations: 7)
Positive-Sense: 
FULLGENOME|fullspan    1
GENE|1AB               1

Negative-Sense: 
FULLGENOME|fullspan    6
GENE|1AB               3
GENE|S                 1
GENE|3a                1
GENE|N                 1
GENE|9b                1
UTR|5putr              1



---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 7
number_of_miRNAs_in_cluster: 1
miR: hsa_miR_548ba (total uniq predicted binding locations: 7)
Positive-Sense: 
FULLGENOME|fullspan    4
GENE|1AB               4

Negative-Sense: 
FULLGENOME|fullspan    3
GENE|1AB               3



---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 7
number_of_miRNAs_in_cluster: 1
miR: hsa_miR_3135a (total uniq predicted binding locations: 7)
Positive-Sense: 
FULLGENOME|fullspan    3
GENE|1AB               2
GENE|S                 1

Negative-Sense: 




---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 2
number_of_miRNAs_in_cluster: 1
miR: hsa_miR_5690 (total uniq predicted binding locations: 2)
Negative-Sense: 
FULLGENOME|fullspan    2
GENE|S                 1
GENE|3a                1



---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 2
number_of_miRNAs_in_cluster: 1
miR: hsa_miR_3925_3p (total uniq predicted binding locations: 2)
Positive-Sense: 
FULLGENOME|fullspan    1
GENE|1AB               1

Negative-Sense: 
FULLGENOME|fullspan    1
GENE|N                 1



---------------- Cluster Start -----------------------------------
unique_predicted_target_locations: 2
number_of_miRNAs_in_cluster: 1
miR: hsa_miR_9983_3p (total uniq predicted binding locations: 2)
Positive-Sense: 
FULLGENOME|fullspan    1
GENE|N                 1

Negative-Sense: 
FULLGENOME|fullspan    1
GENE|3a                1
GENE|3b                1



-----